# Params for Simple_RNN

$params = cell\times input\_size + cell^2$

$cell^2$ means cells are fullly connected to themselves   meaning?


In [0]:
%tensorflow_version 1.x

In [0]:
# colab.research.google.com
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
import numpy as np


In [0]:
def get_bit(n, bit):
    a = []
    for i in range(bit):
        a.append(n & 1)
        n >>= 1
    return a

In [0]:
import random

def train_generator():
    while True:
      r1 = random.randrange(1024)
      r2 = random.randrange(1024)
      bit = max(r1.bit_length(), r2.bit_length(), ) + 1
      a = get_bit(r1, bit)
      b = get_bit(r2, bit)
      c = get_bit(r1+r2, bit)
      x = np.array([np.array([a[i], b[i]]) for i in range(bit)])
      y = np.array([np.array([c[i]]) for i in range(bit)])
      x = x.reshape(1, bit, 2)
      y = y.reshape(1, bit, 1)
      yield x, y


# Variable Length
output for every input

In [5]:
cell = 16
input_size = 2

model = keras.Sequential([
        SimpleRNN(cell, activation='relu', use_bias=False, input_shape=(None, input_size), return_sequences=True),
        TimeDistributed(Dense(1, activation='relu', use_bias=False))
])
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, None, 16)          288       
_________________________________________________________________
time_distributed (TimeDistri (None, None, 1)           16        
Total params: 304
Trainable params: 304
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam', #tf.keras.optimizers.RMSprop(learning_rate=0.1),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [7]:
model.fit(train_generator(), epochs=20, steps_per_epoch=1000)

Epoch 1/20
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
1000/1000 [==============================] - 5s 5ms/step - loss: 3.1286 - acc: 0.5173
Epoch 2/20
1000/1000 [==============================] - 5s 5ms/step - loss: 1.2667 - acc: 0.4999
Epoch 3/20
1000/1000 [==============================] - 5s 5ms/step - loss: 1.1610 - acc: 0.5469
Epoch 4/20
1000/1000 [==============================] - 5s 5ms/step - loss: 1.0553 - acc: 0.6027
Epoch 5/20
1000/1000 [==============================] - 5s 5ms/step - loss: 0.9944 - acc: 0.6442
Epoch 6/20
1000/1000 [==============================] - 5s 5ms/step - loss: 0.7476 - acc: 0.7247
Epoch 7/20
1000/1000 [==============================] - 5s 5ms/step - loss: 0.3755 - acc: 0.8280
Epoch 8/20
1000/1000 [==============================] - 5s 5ms/step - loss: 0.2559 - acc: 0.9053
Epoch 9/20
1000/1000 [==============================] - 5s 5ms/step - loss: 0.1179 - acc: 0.9649
Epoch 10/20
1000/1000 [==========

# Test

In [8]:
x, y = next(train_generator())
print(x)
model.predict_classes(x)

[[[0 0]
  [1 1]
  [0 1]
  [1 0]
  [0 1]
  [0 1]
  [0 1]
  [0 0]
  [1 0]
  [0 0]]]


array([[[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0]]], dtype=int32)

In [0]:
def add(a, b):
  bit = max(a.bit_length(), b.bit_length(), ) + 1
  a = get_bit(a, bit)
  b = get_bit(b, bit)
  x = np.array([np.array([a[i], b[i]]) for i in range(bit)])
  x = x.reshape(1, bit, 2)
  a = model.predict_classes(x)
  a = np.flip(a.reshape(a.size))
  return int("".join(str(x) for x in list(a)), 2) 

In [10]:
count = 0
for _ in range(1000):
  a = random.randrange(1000000000000)
  b = random.randrange(1000000000000)
  if add(a, b) != a + b:
    count += 1
print(count)

0
